# ArcGIS Server Services Audit

Description: This script goes through all of the Services on an ArcGIS Server and writes the parameters to Excel.
 
Code inspired by: AGOServiceAuditPythonAPI_SQL.py
 
Created on: 2/13/2018
 
Purpose: You supply the Server URL and the Administrator Account User and Password then the script goes through all of the Services on an ArcGIS Server and writes the parameters to Excel.
 
Authored by: Brandon Longenberger & Joe Guzi
 
Previous Production Date: 9/9/2024

Production Date: 9/10/2024

Notes:
- 8/13/2024 --- updated with Audit Template
- 9/9/2024 - updated the code to account for shared instances
- 9/10/2024 - updated the code to account for hosted services. Also changed the url input

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis.server import Server
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Section Complete!")

# Step 2: Functions

In [ ]:
print("Begin Function Section.")
def ParameterExists(Set, Parameter):
    try:
        Variable = Set[Parameter] 
    except:
        Variable = "No Value"
    return Variable
def numParameterExists(Set, Parameter):
    try:
        Variable = Set[Parameter] 
    except:
        Variable = 0
    return Variable
def ProviderTranslator(ServiceProvider):
    if ServiceProvider == "ArcObjects11":
        Provider = "ArcGISPro-Dedicated"
    elif ServiceProvider == "ArcObjects":
        Provider = "ArcMap"
    elif ServiceProvider == "DMaps":
        Provider = "ArcGISPro-Shared"
    elif ServiceProvider == "SDS":
        Provider = "Hosted"
    else:
        Provider = ""
    return Provider
def ServerAudit(ServerUrl, ServerUsername, ServerPassword):
    t=""
    ArcGISServer = Server(url=ServerUrl, username=ServerUsername, password=ServerPassword)
    ArcGISServer
    ArcGISServerFolders = ArcGISServer.services.folders
    for ArcGISServerFolder in ArcGISServerFolders:
        ArcGISServerFolderServicesList = ArcGISServer.services.list(folder=ArcGISServerFolder)
        for ArcGISServerService in ArcGISServerFolderServicesList:
            ArcGISServiceJSON = ArcGISServerService.properties
            #print(ArcGISServiceJSON)
            ServiceName = ArcGISServiceJSON.serviceName
            Folder = ArcGISServerFolder
            Type = ArcGISServiceJSON.type
            if Type == "GeometryServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                Extensions = ArcGISServiceJSON.extensions
                FeatureService = "NA"
                KML = "NA"
                WMS = "NA"
                MaxRecords = "NA"
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = "NA"
                JobsDirectory = "NA"
                OutputDirectory = "NA"
                FilePath = "NA"
                JSONProvider = ArcGISServiceJSON.provider
                #print("JSONProvider: " + str(JSONProvider))
                Provider = ProviderTranslator(JSONProvider)
            if Type == "GPServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                Extensions = ArcGISServiceJSON.extensions
                FeatureService = "NA"
                KML = "NA"
                WMS = "NA"
                MaxRecords = "NA"
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = "NA"
                JobsDirectory = ArcGISServiceJSON.properties.jobsDirectory
                OutputDirectory = ArcGISServiceJSON.properties.outputDir
                FilePath = "NA"
                JSONProvider = ArcGISServiceJSON.provider
                #print("JSONProvider: " + str(JSONProvider))
                Provider = ProviderTranslator(JSONProvider)
            if Type == "MapServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                Extensions = ArcGISServiceJSON.extensions
                for ExtensionType in Extensions:
                    ExtensionTypeName = ExtensionType.typeName
                    if ExtensionTypeName == "FeatureServer":
                        FeatureService = ExtensionType.enabled
                    if ExtensionTypeName == "KmlServer":
                        KML = ExtensionType.enabled
                    if ExtensionTypeName == "WMSServer":
                        WMS = ExtensionType.enabled
                MaxRecords = numParameterExists(ArcGISServiceJSON.properties, "maxRecordCount") # we touched this
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = ParameterExists("ArcGISServiceJSON.properties", "cacheDir") #ArcGISServiceJSON.properties.cacheDir
                JobsDirectory = ParameterExists("ArcGISServiceJSON.properties", "jobsDirectory")  #ArcGISServiceJSON.properties.jobsDirectory
                OutputDirectory = ParameterExists("ArcGISServiceJSON.properties","outputDir") #ArcGISServiceJSON.properties.outputDir
                FilePath = ParameterExists(ArcGISServiceJSON.properties, "filePath")
                JSONProvider = ArcGISServiceJSON.provider
                #print("JSONProvider: " + str(JSONProvider))
                Provider = ProviderTranslator(JSONProvider)
            if Type == "FeatureServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                FeatureService = "TRUE"
                KML = ""
                WMS = ""
                MaxRecords = numParameterExists(ArcGISServiceJSON.properties, "maxRecordCount") # we touched this
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = ParameterExists(ArcGISServiceJSON.properties, "cacheDir") 
                JobsDirectory = ParameterExists(ArcGISServiceJSON.properties, "jobsDirectory")  
                OutputDirectory = ParameterExists(ArcGISServiceJSON.properties,"outputDir")
                FilePath = ParameterExists(ArcGISServiceJSON.properties, "filePath")
                JSONProvider = ArcGISServiceJSON.provider
                Provider = ProviderTranslator(JSONProvider)
            if Type == "ImageServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                FeatureService = "NA"
                KML = "NA"
                Extensions = ArcGISServiceJSON.extensions
                for ExtensionType in Extensions:
                    ExtensionTypeName = ExtensionType.typeName
                    if ExtensionTypeName == "WMSServer":
                        WMS = ExtensionType.enabled
                MaxRecords = "NA"
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = "NA"
                JobsDirectory = "NA"
                OutputDirectory= ArcGISServiceJSON.properties.outputDir
                FilePath = "NA"
                JSONProvider = ArcGISServiceJSON.provider
                #print("JSONProvider: " + str(JSONProvider))
                Provider = ProviderTranslator(JSONProvider)
            if Type == "GeocodeServer":
                Status = ArcGISServiceJSON.configuredState
                if ArcGISServiceJSON.provider == "DMaps":
                    MinInstances = 0
                    MaxInstances = 0
                elif ArcGISServiceJSON.provider == "SDS":
                    MinInstances = 0
                    MaxInstances = 0
                else:
                    MinInstances = ArcGISServiceJSON.minInstancesPerNode
                    MaxInstances = ArcGISServiceJSON.maxInstancesPerNode
                Extensions = ArcGISServiceJSON.extensions
                FeatureService = "NA"
                KML = "NA"
                WMS = "NA"
                MaxRecords = "NA"
                Cluster = ArcGISServiceJSON.clusterName
                CacheDirectory = "NA"
                JobsDirectory = "NA"
                OutputDirectory= ArcGISServiceJSON.properties.outputDir
                FilePath = "NA"
                JSONProvider = ArcGISServiceJSON.provider
                #print("JSONProvider: " + str(JSONProvider))
                Provider = ProviderTranslator(JSONProvider)
            t += str(ServerUrl) + ","
            t += str(ServiceName) + ","
            t += str(Folder) + ","
            t += str(Type) + ","
            t += str(Status) + ","
            t += str(MinInstances) + ","
            t += str(MaxInstances) + ","
            t += str(FeatureService) + ","
            t += str(KML) + ","
            t += str(WMS) + ","
            t += str(MaxRecords) + ","
            t += str(Cluster) + ","
            t += str(CacheDirectory) + ","
            t += str(JobsDirectory) + ","
            t += str(OutputDirectory) + ","
            t +=  str(FilePath) + ","
            t +=  str(Provider) + "\n"
    return t

print("Function Section Complete!")

# Step 3: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
# Connection Variables
print("ArcGIS Server Admin URL Examples:")
print("Standalone ArcGIS Server: https://ServerName.DomainName:portnumber/arcgis/admin")
print("Federated ArcGIS Server: https://ServerName.DomainName/webadaptor/admin")
ArcGISServer = input("Enter the ArcGIS Server Admin URL: ")
ArcGISServerAdministratorUser = input("Enter the ArcGIS Server Administrator Username: ")
ArcGISServerAdministratorPassword = getpass.getpass('ArcGIS Server Administrator Password: ')
print("Auditing " + str(ArcGISServer) + " .....")
Audit = ServerAudit(ArcGISServer, ArcGISServerAdministratorUser, ArcGISServerAdministratorPassword)
print("Execute Code Section Complete!") 

# Step 4: Write the Audit Results to Excel

In [ ]:
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("ServerName,ServiceName,Folder,Type,Status,Min Instances,Max Instances,FeatureService,kml,wms,Max Records,Cluster,Cache Directory,Jobs Directory,Output Directory,File Path, Published From" + "\n")
    output.writelines(Audit)
    print("Wrtie data To CSV File Complete!")